# Entrainement du modèle d'identification du grade de la route

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### 1. Chargement du dataframe contenant les données d'entrainement

In [4]:
# Téléchargement et extraction des fichiers

!wget -O DataFrameGlobal.zip -q http://62.210.208.36/DataFrameGlobal.zip
!unzip /content/DataFrameGlobal.zip

Archive:  /content/DataFrameGlobal.zip
  inflating: DataFrameGlobal.csv     


In [5]:
# Création du DataFrame
df = pd.read_csv("/content/DataFrameGlobal.csv")

In [6]:
df

,Temps (s),H_v3.3_p2.0,B_v4.6_p2.0,C_v5.3_p1.5,E_v4.9_p1.9,F_v5.0_p1.6,D_v4.7_p1.9,H_v1.7_p2.3,G_v3.6_p2.3,A_v3.6_p2.2,...,H_v4.8_p2.4,C_v1.9_p2.4,B_v5.0_p2.2,C_v4.2_p2.3,B_v1.9_p2.2,B_v3.4_p1.5,H_v5.5_p1.5,A_v3.5_p2.1,H_v2.3_p1.6,D_v3.9_p1.6
0,0.01,9.817928,9.810112,9.810359,9.811507,9.812488,9.810424,9.812690,9.813991,9.810062,...,9.825546,9.810066,9.810155,9.810306,9.810040,9.810088,9.822273,9.810050,9.813930,9.810286
1,0.02,9.817914,9.810112,9.810358,9.811503,9.812481,9.810423,9.812688,9.813983,9.810062,...,9.825504,9.810066,9.810155,9.810305,9.810040,9.810088,9.822236,9.810050,9.813925,9.810286
2,0.03,9.817903,9.810112,9.810357,9.811500,9.812476,9.810422,9.812686,9.813977,9.810062,...,9.825475,9.810066,9.810154,9.810304,9.810040,9.810088,9.822207,9.810049,9.813921,9.810285
3,0.04,9.817886,9.810111,9.810356,9.811495,9.812468,9.810421,9.812683,9.813968,9.810062,...,9.825428,9.810066,9.810154,9.810304,9.810040,9.810088,9.822165,9.810049,9.813915,9.810285
4,0.05,9.817870,9.810111,9.810355,9.811491,9.812461,9.810420,9.812680,9.813959,9.810062,...,9.825381,9.810066,9.810153,9.810303,9.810040,9.810087,9.822123,9.810049,9.813910,9.810284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9.96,9.880820,9.810467,9.810553,9.813605,9.813951,9.806159,9.795998,9.828026,9.810280,...,9.713050,9.810871,9.810246,9.810015,9.810534,9.765802,9.832789,9.809665,9.876172,9.811255
996,9.97,21.818193,9.810466,9.810552,9.813594,9.813940,9.806170,9.796006,29.916127,10.122634,...,9.713354,9.810354,9.810246,9.810015,9.810217,9.591306,9.832711,9.604913,9.876083,9.811252
997,9.98,9.783262,9.810465,9.810550,9.813583,9.813927,9.806180,9.796022,9.813416,9.809976,...,9.713613,9.810342,9.810245,9.810015,9.810210,9.810794,9.832632,9.810097,9.876026,9.451535
998,9.99,9.783406,9.810463,9.810548,9.813572,9.813915,9.806192,9.796037,9.782474,9.809572,...,9.713924,9.810339,9.810244,9.815183,9.810208,9.810795,9.832554,9.810116,9.875933,9.811735


Affichage de quelques séries :

In [7]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df),len(df)+1),y=df[df.columns[1]], line=dict(color='blue', width=1),name=df.columns[1]))
fig.add_trace(go.Scatter(x=np.linspace(0,len(df),len(df)+1),y=df[df.columns[2]], line=dict(color='red', width=1),name=df.columns[2]))
fig.add_trace(go.Scatter(x=np.linspace(0,len(df),len(df)+1),y=df[df.columns[3]], line=dict(color='black', width=1),name=df.columns[3]))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

### 2. Création du dataset d'entrainement et de test

**2.1. Création des datasets global (avec les données d'entrainement et de test)**

Le dataset d'entrainement contient des milliers de séquences de 30 secondes, chacune étant identifiée comme appartenant à un grade spécifique de la route (A,B,C,D,E,F,G ou H) : 

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/raw/main/VibrationRoute/ExtractionAccel.jpg" width=800></center>

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/raw/main/VibrationRoute/GradesRoutes.jpg" width=600></center>

Pour coder les labels, on utilise le format one_hot avec la fonction [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical):

In [8]:
from keras.utils import to_categorical

grades = [0,1,2,3,4,5,6,7]

codes_one_hot = to_categorical(grades)
print(codes_one_hot)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


La structure du dataset est la suivante (si batch_size = 1) :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/raw/main/VibrationRoute/ContenuDataset2.jpg" width=900></center>

In [9]:
# Extraction des quantités annexes à partir des données d'accélération
def get_annexes(serie):
  # Variance
  variance = tf.math.reduce_variance(serie)
  # Ecart type
  std = tf.math.reduce_std(serie)
  # max
  max = tf.math.reduce_max(serie)
  # min
  min = tf.math.reduce_min(serie)

  return tf.convert_to_tensor([variance, std, max, min], dtype=tf.float32)

In [10]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle
# au format X(X1,X2,...X300), Y, Z
# X sont les échantillons aux pas de 0.1s (100 échantillons = 10 secondes)
# Y sont les entrées annexes (vitesse et quantités extraites des données d'accélération)
# Z sont les labels (grades de la route) au format one_hot : A=0, B=1, ..., H=7

def prepare_dataset_XY(serie, label, vitesse, taille_fenetre, buffer_melange):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre, shift=taille_fenetre, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre))
  dataset = dataset.map(lambda x: (x/tf.math.reduce_mean(x), tf.concat([vitesse,get_annexes(x)],0), codes_one_hot[label]))
  return dataset

In [11]:
def get_label(grade_route):
    if grade_route == 'A':
      label = 0
    elif grade_route == 'B':
      label = 1
    elif grade_route == 'C':
      label = 2
    elif grade_route == 'D':
      label = 3
    elif grade_route == 'E':
      label = 4
    elif grade_route == 'F':
      label = 5
    elif grade_route == 'G':
      label = 6
    elif grade_route == 'H':
      label = 7
    return tf.convert_to_tensor(label)

In [12]:
taille_fenetre = 100*10
batch_size = 32
ratio_entrainement = 0.8

# Création du dataset initial
serie = df[df.columns[1]].values

grade_route = df.columns[1].split("_")[0]
vitesse = tf.convert_to_tensor([float(df.columns[1].split("_")[1].split("v")[1])])
label = get_label(grade_route)
dataset = prepare_dataset_XY(serie,label,vitesse, taille_fenetre,10000)

# Concaténation avec les datasets suivants
from tqdm import tqdm
for i in tqdm(range(len(df.columns))):
  colonne = df.columns[i]
  if i > 1:
    # Extraction des labels
    grade_route = colonne.split("_")[0]
    vitesse = tf.convert_to_tensor([float(colonne.split("_")[1].split("v")[1])])
    label = get_label(grade_route)
    
    # Création de la série
    serie = df[colonne].values

    # Création du dataset
    dataset_ = prepare_dataset_XY(serie,label,vitesse,taille_fenetre,10000)
    dataset = dataset.concatenate(dataset_)

# Création des batchs
dataset = dataset.batch(batch_size,drop_remainder=True)

100%|██████████| 695/695 [00:33<00:00, 21.01it/s]


**2.2. Séparation des données d'entrainement et de tests**

Format des dataset : [(serie,annexes,label), ...]

In [13]:
ratio_entrainement = 0.8
dataset_ent, dataset_test = tf.keras.utils.split_dataset(dataset, left_size=ratio_entrainement)

**2.3. Réorganisation des datasets**

Format final des dataset : [((serie,annexes),label), ...]

In [14]:
dataset_ent = dataset_ent.map(lambda x,y,z : ((x,y),z))
dataset_test = dataset_test.map(lambda x,y,z : ((x,y),z))

In [ ]:
i = 0
for element in dataset_ent:
  i=i+1
  if i > 1:
    break
  print(element)

### 3. Mise en place du modèle

**3.1. Création du modèle à base de réseaux de convolution**

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/raw/main/VibrationRoute/StructureModele.jpg" width=600></center>

In [18]:
from keras.layers.merging.concatenate import concatenate

entree_serie = tf.keras.layers.Input(shape=(taille_fenetre), name="First")
entree_annexes = tf.keras.layers.Input(shape=(5), name="Second")

# Réseau de convolution
conv1 = keras.layers.Conv1D(filters=64, kernel_size=7, padding="same")(tf.expand_dims(entree_serie,axis=-1))
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.ReLU()(conv1)
conv1 = keras.layers.MaxPool1D()(conv1)

conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.ReLU()(conv2)

conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)

conv4 = keras.layers.Add()([conv1,conv3])
conv4 = keras.layers.ReLU()(conv4)
conv4 = keras.layers.MaxPool1D()(conv4)
conv4 = keras.layers.Flatten()(conv4)

# Prise en compte des entrées annexes
concat = keras.layers.concatenate([conv4,entree_annexes])

# Construction du modèle
output_layer = keras.layers.Dense(1024, activation="relu")(concat)
output_layer = keras.layers.Dense(512, activation="relu")(concat)
output_layer = keras.layers.Dense(8, activation="softmax")(output_layer)


model = tf.keras.Model(inputs=[entree_serie,entree_annexes],outputs=output_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 First (InputLayer)             [(None, 1000)]       0           []                               
                                                                                                  
 tf.expand_dims_2 (TFOpLambda)  (None, 1000, 1)      0           ['First[0][0]']                  
                                                                                                  
 conv1d_3 (Conv1D)              (None, 1000, 64)     512         ['tf.expand_dims_2[0][0]']       
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 1000, 64)    256         ['conv1d_3[0][0]']               
 rmalization)                                                                               

**3.2. Définition de l'optimiseur et de la fonction d'erreur**

On utilise l'optimiseur Adam ainsi qu'une fonction d'erreur de type [CategoricalCrossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class).

L'ensemble des fonctions d'erreur utilisables avec Keras se trouve ici : https://keras.io/api/losses/

La fonction d'erreur que nous utilisons permet de gérer les labels multiples qui ont été codés en représentation "one_hot".

In [19]:
# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compile le modèle
model.compile(loss="categorical_crossentropy", optimizer=optimiseur, metrics=['accuracy'])

**3.3. Entrainement du modèle**

In [20]:
historique = model.fit(dataset_ent, validation_data = dataset_test, epochs=300, batch_size=batch_size)

Epoch 1/300
17/17 [==============================] - 10s 382ms/step - loss: 6.4932 - accuracy: 0.3996 - val_loss: 2.4869 - val_accuracy: 0.3507
Epoch 2/300
17/17 [==============================] - 5s 271ms/step - loss: 1.0465 - accuracy: 0.6729 - val_loss: 1.8543 - val_accuracy: 0.3881
Epoch 3/300
17/17 [==============================] - 5s 311ms/step - loss: 0.5877 - accuracy: 0.7695 - val_loss: 2.8295 - val_accuracy: 0.2090
Epoch 4/300
17/17 [==============================] - 5s 307ms/step - loss: 0.4918 - accuracy: 0.8197 - val_loss: 3.9587 - val_accuracy: 0.1343
Epoch 5/300
17/17 [==============================] - 6s 374ms/step - loss: 0.3618 - accuracy: 0.8792 - val_loss: 4.7667 - val_accuracy: 0.1269
Epoch 6/300
17/17 [==============================] - 5s 278ms/step - loss: 0.3111 - accuracy: 0.8699 - val_loss: 5.4715 - val_accuracy: 0.1343
Epoch 7/300
17/17 [==============================] - 6s 349ms/step - loss: 0.2710 - accuracy: 0.9182 - val_loss: 5.8120 - val_accuracy: 0.126

KeyboardInterrupt: ignored

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
model.evaluate(dataset_test)